## Variational Quantum Boltzmann Machine - Qiskit Implementation 

### Imports

In [20]:
# General Imports
import numpy as np
from scipy.linalg import expm

# Circuit Imports
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit import Parameter

# Operator Imports
from qiskit.aqua.operators import I, Z, StateFn, CircuitStateFn, SummedOp
from qiskit.aqua.operators.gradients import NaturalGradient

# Additional Imports
from qiskit.quantum_info import state_fidelity, partial_trace, Statevector
from qiskit.aqua.components.optimizers import SPSA

## Gibbs state preparation

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [2]:
# Temperature
k_BT = 5

# Evolution time
t =  1/(2*k_BT)

# Define the model Hamiltonian 
H = SummedOp([0.3 * Z^Z, 0.2 * Z^I,  0.5 * I^Z]) ^ I^I

### Define the system parameters and initialize an Ansatz

$\rho^{Gibbs} = \frac{e^H/{k_BT}}{Z}$

In [3]:
# Instantiate the model ansatz
depth = 1
ansatz = RealAmplitudes(4, reps=depth, entanglement = 'sca')
qr = ansatz.qregs[0]
for i in range(int(len(qr)/2)):
    ansatz.cx(qr[i], qr[i+int(len(qr)/2)])
    
# Initialize the Ansatz parameters
param_values_init = np.zeros(H.num_qubits * (depth + 1))
for j in range(H.num_qubits * depth, int(len(param_values_init)-H.num_qubits/2)):
    param_values_init[int(j)] = np.pi/2.
    

#### Initial State

The Ansatz $|\psi\left(\omega\left(\tau\right)\right)\rangle$ is initialized such that the first two qubits are in a maximally mixed state.

In [4]:
print('Initial parameters ', param_values_init)

# Initial State

print('\n Circuit ', ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))))

print('\n Full statevector ', CircuitStateFn(ansatz.assign_parameters \
                                          (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data)

print('\n Maximally mixed state', partial_trace(CircuitStateFn(ansatz.assign_parameters\
                        (dict(zip(ansatz.ordered_parameters, param_values_init)))).eval().primitive.data, [0, 1]).data)

Initial parameters  [0.         0.         0.         0.         1.57079633 1.57079633
 0.         0.        ]

 Circuit       ┌───────┐┌───┐     ┌──────────┐                               
q_0: ┤ RY(0) ├┤ X ├──■──┤ RY(pi/2) ├───────────────────────■───────
     ├───────┤└─┬─┘┌─┴─┐└──────────┘┌──────────┐           │       
q_1: ┤ RY(0) ├──┼──┤ X ├─────■──────┤ RY(pi/2) ├───────────┼────■──
     ├───────┤  │  └───┘   ┌─┴─┐    └──────────┘┌───────┐┌─┴─┐  │  
q_2: ┤ RY(0) ├──┼──────────┤ X ├─────────■──────┤ RY(0) ├┤ X ├──┼──
     ├───────┤  │          └───┘       ┌─┴─┐    ├───────┤└───┘┌─┴─┐
q_3: ┤ RY(0) ├──■──────────────────────┤ X ├────┤ RY(0) ├─────┤ X ├
     └───────┘                         └───┘    └───────┘     └───┘

 Full statevector  [0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j
 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j]

 Maximally mixed state [[0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.

#### Let's define the target observable consisting of the Ansatz and the Hamiltonian

$$ \langle \psi\left(\omega\left(\tau\right)\right)|H|\psi\left(\omega\left(\tau\right)\right)\rangle $$

In [6]:
# Define the Hamiltonian as observable w.r.t. the wavefunction generated by the Ansatz   
# Use statevector simulation
ansatz_op = CircuitStateFn(ansatz)
op = ~StateFn(H) @ ansatz_op

print('\n Energy expectation value of the initial state ', op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values_init))).eval())


 Energy expectation value of the initial state  (1.67e-16+0j)


#### Target state

$\rho^{target} = \frac{e^{H\otimes I}/{k_BT}}{Z}$

In [24]:
# Compute the density matrix corresponding to the target Gibbs state
h_mat = H.to_matrix()
gibbs_target = expm(-h_mat*t) / np.trace(expm(-h_mat*t))
gibbs_target = partial_trace(gibbs_target, [0, 1]).data

print('Target state ', gibbs_target)

Target state  [[0.22578687+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.26496334+0.j 0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.24953308+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.25971672+0.j]]


### Define the parameter propagation rule according to McLachlan's variational principle

In [8]:
def get_gibbs_state_params(op, ansatz, param_values, time_steps):
    def NGconvert(
            operator, 
            params,
            method):
        r"""
        Args:
            operator: The operator we are taking the gradient of
            params: The parameters we are taking the gradient with respect to.
            method: The method used to compute the state/probability gradient. Can be either
                ``'param_shift'`` or ``'lin_comb'`` or ``'fin_diff'``. Deprecated for observable gradient.
            regularization: Use the following regularization with an lstsq method to solve the underlying SLE
                Can be either None or ``'ridge'`` or ``'lasso'`` or ``'perturb_diag'``
                ``'ridge'`` and ``'lasso'`` use an automatic optimal parameter search
                If regularization is None but the metric is ill-conditioned or singular then a lstsq solver is
                used without regularization
            approx: Which approximation of the QFI to use: [None, 'diagonal', 'block_diagonal']

        Returns:
            An operator whose evaluation yields the NaturalGradient.

        Raises:
            ValueError: If ``params`` contains a parameter not present in ``operator``.
        """
        from qiskit.aqua.operators.gradients import QFI, Gradient
        from qiskit.aqua.operators import ComposedOp, ListOp
        if (not isinstance(operator, ComposedOp)) or (not isinstance(operator.oplist[-1], CircuitStateFn)):
            raise AquaError('Please make sure that the operator for which you want to compute the natural gradient'
                            ' represents an expectation value and that the quantum '
                            'state is given as CircuitStateFn.')
        grad = Gradient().convert(operator, params, method)
        metric = QFI().convert(operator[-1], params) * 0.25

        def combo_fn(x):
            c = x[0]
            a = x[1]


            try:
                nat_grad = np.linalg.solve(a, c)
            except np.LinAlgError:
                nat_grad = np.linalg.lstsq(a, c)
#             print(nat_grad)
            return nat_grad

        return ListOp([grad, metric], combo_fn=combo_fn)

    # Convert the operator that holds the Hamiltonian and ansatz into a NaturalGradient operator 
#     nat_grad = NaturalGradient.convert(op, ansatz.ordered_parameters, method = 'lin_comb')
    nat_grad = NGconvert(op, ansatz.ordered_parameters, method = 'lin_comb')

    # Propagate the Ansatz parameters step by step according to the explicit Euler method
    for step in time_steps:
        param_dict = dict(zip(ansatz.ordered_parameters, param_values))
        nat_grad_result = nat_grad.assign_parameters(param_dict).eval()
#         print('param values',  param_values)
#         print('nat_grad_result', nat_grad_result)
        param_values = list(np.subtract(param_values, t/num_time_steps * np.real(nat_grad_result)))
#         param_values -= t/num_time_steps * nat_grad_result
    return param_values


### Run the parameter propagation

In [9]:
# Define the discretization grid of the time steps
num_time_steps = 10
time_steps = np.linspace(0, t, num_time_steps)
param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    
print('Final parameter values', param_values)

Final parameter values [3.4000000000000004e-18, 7.880000000000001e-18, 5.600000000000007e-19, 3.600000000000008e-19, 1.7657540891223833, 1.6401347143700085, 7.200000000000002e-18, -1.959999999999999e-18]


#### Check the fidelity between trained and target state

In [26]:
# Compute the density matrix corresponding to the final Gibbs state    
gibbs_state = Statevector.from_instruction(ansatz.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))))
gibbs_state = partial_trace(gibbs_state, [0, 1])
print('Gibbs state ', np.around(gibbs_state.data, 3))

print('Target state ', np.around(gibbs_target.data, 3))

# Evaluate the fidelity between the trained and the target state
fidelity = state_fidelity(np.around(gibbs_target.data, 3), np.around(gibbs_state.data, 3), validate=False)

print('Fidelity between trained and target state ', fidelity)

Gibbs state  [[ 0.188+0.j  0.   +0.j -0.   +0.j  0.   +0.j]
 [ 0.   +0.j  0.278+0.j -0.   +0.j -0.   +0.j]
 [-0.   +0.j -0.   +0.j  0.216+0.j  0.   +0.j]
 [ 0.   +0.j -0.   +0.j  0.   +0.j  0.319+0.j]]
Target state  [[0.226+0.j 0.   +0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.265+0.j 0.   +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.25 +0.j 0.   +0.j]
 [0.   +0.j 0.   +0.j 0.   +0.j 0.26 +0.j]]
Fidelity between trained and target state  0.995845147830691


## Train a generative QBM 
More explicitly, we train here a fully visible, diagonal, generative QBM using gradient-free optimization.

### Initialize a parameterized Hamiltonian and the target PDF

In [37]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = SummedOp([a * Z^Z, b * Z^I, c * I ^ Z]) ^ I ^ I

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

#### Define the loss function and the optimizer

In [44]:
# Define the loss function
def loss(H_coeffs):
    op = ~StateFn(H.assign_parameters(dict(zip([a, b, c], H_coeffs)))) @ ansatz_op                                 
    param_values = get_gibbs_state_params(op, ansatz, param_values_init, time_steps)
    p_qbm = ansatz_op.assign_parameters(dict(zip(ansatz.ordered_parameters, param_values))).eval().primitive
    p_qbm = np.diag(partial_trace(p_qbm, [0, 1]).data)
    print(p_qbm)
    print(p_target)
    return -np.sum(np.multiply(p_target, np.log(p_qbm)))

optimizer = SPSA(maxiter =  100)

### Train the QBM

In [ ]:
result = optimizer.optimize(3, loss, initial_point = [4. , -0.1, -2.])
print(result)

In [ ]:
a = Parameter('a')    
b = Parameter('b')
c = Parameter('c')  

# Define the model Hamiltonian with parameters
H = (a * Z^Z + b * Z^I - c * I ^ Z) ^ I ^ I

# Define the target PDF
p_target = [0.5, 0, 0, 0.5] 

# Use statevector simulation
ansatz_op = CircuitStateFn(qc = ansatz)
p_qbm = ansatz_op.assign_parameter(dict(zip(ansatz.ordered_parameters, param_values))).eval()


loss = -np.sum(np.multiply(p_target, np.log(p_qbm)))

domega_p_qbm = Gradient().convert(op = ansatz_op, params = ansatz.ordered_parameters, method = 'lin_comb')

qfi = QFI().convert(op = ansatz_op, params = ansatz.ordered_parameters)
from  qiskit.aqua.operator.gradients.gradient.natural_gradient import regularized_sle_solver

dH_omega = regularized_sle_solver(qfi * 0.25, dH_C - dH_A*nat_grad_result, regularization = 'ridge')

dH_p_qbm = domega_p_qbm * dH_omega

dH_loss = -np.tensordot(np.divide(dH_p_qbm, p_qbm), [p_target], axes=1)